<a href="https://colab.research.google.com/github/zakee14910/projectgis/blob/main/02_Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install earthengine-api geemap
!pip install rasterio

# Note: Many packages like numpy, pandas, matplotlib, scikit-learn,
# opencv-python, and tensorflow are pre-installed in Colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 59.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define the base directory for your project
import os
import sys
base_dir = '/content/drive/MyDrive/coconut_plantation_research'

# Add the code directory to the Python path
sys.path.append(f"{base_dir}/code")

# Import Earth Engine and initialize
import ee
try:
  ee.Initialize(project='researchmsc')
  print('Earth Engine already initialized')
except:
  ee.Authenticate()
  ee.Initialize(project='researchmsc')
  print('Earth Engine initialized')

# Import custom modules
from study_areas import get_study_area_geometry, study_areas
from landsat_collection import get_landsat_collection, merge_landsat_collections
from sentinel_collection import get_sentinel_collection, prepare_sentinel_collection
from annual_composites import create_annual_composites, calculate_vegetation_indices
from download_imagery import download_sample_images, visualize_rgb, visualize_ndvi

import geemap
import matplotlib.pyplot as plt


Mounted at /content/drive
Earth Engine initialized


Process a single study area:

In [ ]:
# Select a study area to process
area_name = 'Minuwangoda'  # You can change this to any area in study_areas
print(f"Processing {area_name}...")

# Create area-specific directories
area_raw_dir = os.path.join(base_dir, "data/raw", area_name)
area_processed_dir = os.path.join(base_dir, "data/processed", area_name)
os.makedirs(area_raw_dir, exist_ok=True)
os.makedirs(area_processed_dir, exist_ok=True)

# Get area geometry
area_geometry = get_study_area_geometry(area_name)

# Display the study area on a map
Map = geemap.Map()
Map.centerObject(area_geometry, 12)
Map.addLayer(area_geometry, {'color': 'red'}, f'{area_name} Study Area')
Map


Processing Minuwangoda...


Map(center=[7.101945129444218, 79.99544014334782], controls=(WidgetControl(options=['position', 'transparent_b…

Collect Landsat imagery:

In [ ]:
# Collect Landsat imagery
print("Collecting Landsat imagery...")
l5, l7, l8, l9 = get_landsat_collection(
    area_geometry,
    '2005-01-01',
    '2023-12-31',
    cloud_cover_max=20
)

# Print collection sizes
print(f"Landsat 5 images: {l5.size().getInfo()}")
print(f"Landsat 7 images: {l7.size().getInfo()}")
print(f"Landsat 8 images: {l8.size().getInfo()}")
print(f"Landsat 9 images: {l9.size().getInfo()}")

# Merge Landsat collections
landsat_merged = merge_landsat_collections(l5, l7, l8, l9)
print(f"Total merged Landsat images: {landsat_merged.size().getInfo()}")


Landsat 5 images: 36
Landsat 7 images: 107
Landsat 8 images: 69
Landsat 9 images: 6
Total merged Landsat images: 218


Collect Sentinel-2 imagery:

In [ ]:
# Collect Sentinel-2 imagery
print("Collecting Sentinel-2 imagery...")
s2 = get_sentinel_collection(
    area_geometry,
    '2015-01-01',
    '2023-12-31',
    cloud_cover_max=20
)

# Print collection size
print(f"Sentinel-2 images: {s2.size().getInfo()}")

# Prepare Sentinel-2 collection
s2_prepared = prepare_sentinel_collection(s2)
print(f"Prepared Sentinel-2 images: {s2_prepared.size().getInfo()}")


Sentinel-2 images: 91
Prepared Sentinel-2 images: 91


Create annual composites:

In [ ]:
# Create Landsat annual composites
print("Creating Landsat annual composites...")
landsat_composites = create_annual_composites(
    landsat_merged,
    2005,
    2023,
    area_geometry
)

# Get available years
landsat_years = landsat_composites.aggregate_array('year').getInfo()
print(f"Landsat composite years: {landsat_years}")

# Create Sentinel-2 annual composites
print("Creating Sentinel-2 annual composites...")
sentinel_composites = create_annual_composites(
    s2_prepared,
    2015,
    2023,
    area_geometry
)

# Get available years
sentinel_years = sentinel_composites.aggregate_array('year').getInfo()
print(f"Sentinel-2 composite years: {sentinel_years}")


Creating Landsat annual composites...
Landsat composite years: [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
Creating Sentinel-2 annual composites...
Sentinel-2 composite years: [2018, 2019, 2020, 2021, 2022, 2023]


Calculate vegetation indices:

In [ ]:
# Calculate vegetation indices for Landsat
print("Calculating vegetation indices for Landsat...")
landsat_with_indices = calculate_vegetation_indices(landsat_composites)

# Calculate vegetation indices for Sentinel-2
print("Calculating vegetation indices for Sentinel-2...")
sentinel_with_indices = calculate_vegetation_indices(sentinel_composites)


Calculating vegetation indices for Landsat...
Calculating vegetation indices for Sentinel-2...


Visualize a sample composite:

In [ ]:
# Visualize a sample Landsat composite
sample_year = 2020  # Choose a year that's available in landsat_years
sample_image = landsat_with_indices.filter(ee.Filter.eq('year', sample_year)).first()

# RGB visualization
rgb_map = visualize_rgb(sample_image)
# Instead of rgb_map.add_title(), set the title when creating the map
rgb_map = geemap.Map(center=rgb_map.center, zoom=rgb_map.zoom) # Create a new map with the same center and zoom
rgb_map.addLayer(sample_image, {'bands': ['red', 'green', 'blue'], 'min': 0, 'max': 3000}, f"Landsat RGB Composite - {area_name} {sample_year}")
#rgb_map.add_title(f"Landsat RGB Composite - {area_name} {sample_year}")
rgb_map

# NDVI visualization
ndvi_map = visualize_ndvi(sample_image)
# Instead of ndvi_map.add_title(), set the title when creating the map
ndvi_map = geemap.Map(center=ndvi_map.center, zoom=ndvi_map.zoom) # Create a new map with the same center and zoom
ndvi_map.addLayer(sample_image.select('NDVI'), {'min': 0, 'max': 1, 'palette': ['blue', 'white', 'green']}, f"Landsat NDVI - {area_name} {sample_year}")
#ndvi_map.add_title(f"Landsat NDVI - {area_name} {sample_year}")
ndvi_map


Map(center=[7.101945129444218, 79.99544014334782], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
# Visualize a sample Sentinel composite
sample_year = 2020  # Choose a year that's available in Sentinel_years
sample_image = sentinel_with_indices.filter(ee.Filter.eq('year', sample_year)).first()

# RGB visualization
rgb_map = visualize_rgb(sample_image)
rgb_map.centerObject(area_geometry, 12) # Center the map to the area of interest
rgb_map.addLayer(ee.Image(1).visualize(min=0, max=1, palette=['white']), {}, 'Background')
# Changed bands to Sentinel-2 bands (B4, B3, B2) -> (red, green, blue) for Sentinel-2
rgb_map.addLayer(sample_image, {'bands': ['red', 'green', 'blue'], 'min': 0, 'max': 3000}, f"Sentinel-2 RGB Composite - {area_name} {sample_year}") # Add the RGB layer with a title to the map
rgb_map


# NDVI visualization
ndvi_map = visualize_ndvi(sample_image)
ndvi_map.centerObject(area_geometry, 12) # Center the map to the area of interest
ndvi_map.addLayer(ee.Image(1).visualize(min=0, max=1, palette=['white']), {}, 'Background')
ndvi_map.addLayer(sample_image.select('NDVI'), {'min': 0, 'max': 1, 'palette': ['blue', 'white', 'green']}, f"Sentinel-2 NDVI - {area_name} {sample_year}") # Add the NDVI layer with a title to the map
ndvi_map

Map(center=[7.101945129444218, 79.99544014334782], controls=(WidgetControl(options=['position', 'transparent_b…

Download sample images:

In [ ]:
# Download sample Landsat images (one every 5 years)
print("Downloading sample Landsat images...")
landsat_dir = os.path.join(area_raw_dir, "landsat")
sample_years = [year for year in range(2005, 2024, 1) if year in landsat_years]

landsat_paths = download_sample_images(
    landsat_with_indices,
    area_geometry,
    landsat_dir,
    sample_years=sample_years,
    prefix="landsat"
)

# Download sample Sentinel-2 images
print("Downloading sample Sentinel-2 images...")
sentinel_dir = os.path.join(area_raw_dir, "sentinel")
sentinel_sample_years = [year for year in [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023] if year in sentinel_years]

sentinel_paths = download_sample_images(
    sentinel_with_indices,
    area_geometry,
    sentinel_dir,
    sample_years=sentinel_sample_years,
    prefix="sentinel"
)


Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/coconut_plantation_research/data/raw/Minuwangoda/landsat/landsat_2005.tif
Downloaded: /content/drive/MyDrive/coconut_plantation_research/data/raw/Minuwangoda/landsat/landsat_2005.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/coconut_plantation_research/data/raw/Minuwangoda/landsat/landsat_2006.tif
Downloaded: /content/drive/MyDrive/coconut_plantation_research/data/raw/Minuwangoda/landsat/landsat_2006.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/coconut_plantation_research/data/raw/Minuwangoda/landsat/landsat_2007.tif
Downloaded: /content/drive/MyDrive/coconut_plantation_research/data/raw/Minuwangoda/landsat/landsat_2007.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/coconut_plantation_research/data/raw/Minuwangoda/landsat/landsat_2008.tif
Downloaded: /content/drive/MyDrive/coconut_plantation_research/data/r

Save the collections for future use:

In [ ]:
# Export the image collections as assets for future use
# This is optional but can save processing time in future notebooks

# Function to export a collection as an asset
def export_collection_to_asset(collection, asset_id, description):
    # Create a list of images with properties
    image_list = collection.toList(collection.size())

    # Get the size
    size = image_list.size().getInfo()

    for i in range(size):
        image = ee.Image(image_list.get(i))
        year = image.get('year').getInfo()

        # Create asset ID
        image_asset_id = f"{asset_id}_{year}"

        # Export the image
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=f"{description}_{year}",
            folder= "coconut_plantation_research/assets",
            fileNamePrefix=f"{description}_{year}",
            scale=30,
            region=image.geometry(),
            maxPixels=1e9
        )

        task.start()
        print(f"Started export task for {description} {year}")

# Export Landsat composites with indices
export_collection_to_asset(
    landsat_with_indices,
    f"landsat_{area_name.lower()}",
    f"landsat_{area_name.lower()}"
)

# Export Sentinel-2 composites with indices
export_collection_to_asset(
    sentinel_with_indices,
    f"sentinel_{area_name.lower()}",
    f"sentinel_{area_name.lower()}"
)

print("Export tasks started. Check the 'Tasks' tab in Google Earth Engine to monitor progress.")


Started export task for landsat_minuwangoda 2005
Started export task for landsat_minuwangoda 2006
Started export task for landsat_minuwangoda 2007
Started export task for landsat_minuwangoda 2008
Started export task for landsat_minuwangoda 2009
Started export task for landsat_minuwangoda 2010
Started export task for landsat_minuwangoda 2011
Started export task for landsat_minuwangoda 2012
Started export task for landsat_minuwangoda 2013
Started export task for landsat_minuwangoda 2014
Started export task for landsat_minuwangoda 2015
Started export task for landsat_minuwangoda 2016
Started export task for landsat_minuwangoda 2017
Started export task for landsat_minuwangoda 2018
Started export task for landsat_minuwangoda 2019
Started export task for landsat_minuwangoda 2020
Started export task for landsat_minuwangoda 2021
Started export task for landsat_minuwangoda 2022
Started export task for landsat_minuwangoda 2023
Started export task for sentinel_minuwangoda 2018
Started export task